# Stable Diffusion 3 ControlNet のサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用しましょう。（利用制限があるのでこまめにインスタンスは削除してください）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#SD3 で必要なモジュールのインストール
!pip install -Uqq diffusers transformers ftfy accelerate bitsandbytes controlnet_aux omegaconf GPUtil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.3 MB/s eta 0:00:00


下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [2]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')


from huggingface_hub import login
from google.colab import userdata
HF_LOGIN = userdata.get('HF_LOGIN')
login(HF_LOGIN)

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/**/colab_SD3ControlNet_sample/SD3ControlNet_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

Mounted at /content/drive
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
/content/drive/MyDrive/colab_zenn_sub/colab_SD3ControlNet_sample
/content/drive/MyDrive/colab_zenn_sub/colab_SD3ControlNet_sample
/content/drive/MyDrive/colab_zenn_sub/colab_SD3ControlNet_sample


In [3]:
#モジュールをimportする
from module.module_sd3c import SD3C
import time

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [4]:
#モデルの設定を行う。


config_text = """
[SD3C]
device = auto
n_steps=28
seed=42
shift = 3.0

model_path = stabilityai/stable-diffusion-3-medium-diffusers
;controlnet_path = InstantX/SD3-Controlnet-Pose
controlnet_path = InstantX/SD3-Controlnet-Canny
;controlnet_path = InstantX/SD3-Controlnet-Tile

guided_scale = 4.0
width = 1024
height = 1024

use_cpu_offload = False
use_text_encoder_3 = True
use_t5_quantization = True
save_latent = False

"""

#やりたいこと
#ネガティブプロンプトの指定

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

生成する画像を決定するプロンプトを指定します。

negative_promptを設定することで，ここに書かれている内容を回避できる。


In [5]:
#読み上げるプロンプトを設定する。


main_prompt = """
Anime style, An anime digital illustration of young woman with striking red eyes, standing outside surrounded by falling snow and cherry blossoms. Her long, flowing silver hair is intricately braided, adorned an accessory made of lace, white tripetal flowers, and small, white pearls, cascading around her determined face. She wears a high-collared, form-fitting white dress with intricate lace details and cut-out sections on the sleeves, adding to the air of elegance and strength. The background features out-of-focus branches dotted with vibrant pink blossoms, set against a clear blue sky speckled with snow and bokeh, creating a dreamlike, serene atmosphere. The lighting casts a soft, almost divine glow on her, emphasizing her resolute expression. The artwork captures her from the waist up. Image is an anime style digital artwork. trending on artstation, pixiv, anime-style, anime screenshot
"""

#main_prompt = """
#Anime style, pixiv
#"""
negative_prompt=""
#negative_prompt="NSFW, nude, naked, porn, ugly"

input_refer_image_path = "./inputs/refer.webp"
output_refer_image_path = "./inputs/refer.png"


In [6]:

sd = SD3C()
sd.prepare_referimage(input_refer_image_path = input_refer_image_path, output_refer_image_path = output_refer_image_path, low_threshold = 100, high_threshold = 200)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

text_encoder_3/config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

transformer/config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

FrozenDict([('num_train_timesteps', 1000), ('shift', 3.0), ('_use_default_values', ['num_train_timesteps'])])


In [7]:

for i in range(5):
      start = time.time()
      image = sd.generate_image(main_prompt, neg_prompt = negative_prompt,image_path = output_refer_image_path, controlnet_conditioning_scale = 0.5)
      print("generate image time: ", time.time()-start)
      image.save("./outputs/SD3C_result_{}.png".format(i))

Token indices sequence length is longer than the specified maximum sequence length for this model (188 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['with intricate lace details and cut - out sections on the sleeves, adding to the air of elegance and strength. the background features out - of - focus branches dotted with vibrant pink blossoms, set against a clear blue sky speckled with snow and bokeh, creating a dreamlike, serene atmosphere. the lighting casts a soft, almost divine glow on her, emphasizing her resolute expression. the artwork captures her from the waist up. image is an anime style digital artwork. trending on artstation, pixiv, anime - style, anime screenshot']


prepipe exists.


Token indices sequence length is longer than the specified maximum sequence length for this model (188 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['with intricate lace details and cut - out sections on the sleeves, adding to the air of elegance and strength. the background features out - of - focus branches dotted with vibrant pink blossoms, set against a clear blue sky speckled with snow and bokeh, creating a dreamlike, serene atmosphere. the lighting casts a soft, almost divine glow on her, emphasizing her resolute expression. the artwork captures her from the waist up. image is an anime style digital artwork. trending on artstation, pixiv, anime - style, anime screenshot']


| ID | GPU | MEM |
------------------
|  0 | 87% | 72% |
| ID | GPU | MEM |
------------------
|  0 | 49% |  1% |


  0%|          | 0/28 [00:00<?, ?it/s]

...

generate image time:  44.50627875328064


  0%|          | 0/28 [00:00<?, ?it/s]

generate image time:  44.764564037323


  0%|          | 0/28 [00:00<?, ?it/s]

generate image time:  44.65323185920715


In [8]:
main_prompt = """
One cute girl posed cute in the park, wearing a red uniform, chestnut hair, short hair, wearing a mini skirt,She is waving at us,anime,pixiv
"""

In [10]:
for i in range(10,11):
      start = time.time()
      image = sd.generate_image(main_prompt, neg_prompt = negative_prompt,image_path = output_refer_image_path, seed = 1897604184, controlnet_conditioning_scale = 0.5)
      print("generate image time: ", time.time()-start)
      image.save("./outputs/SD3C_result_{}.png".format(i))

  0%|          | 0/28 [00:00<?, ?it/s]

generate image time:  45.29654550552368
